# Data Collection
In this notebook, I'll collect data from Harvard Museum.

In [51]:
import pandas as pd
import numpy as np
import requests
import json

import pickle

import time
from PIL import Image

In [12]:
# getting keys
def get_keys(path):
    with open(path) as f: 
        return json.load(f)

In [13]:
path = r"C:\Users\Eunjoo\Documents\.secret"

In [14]:
api_key = get_keys(f"{path}/harvard_mus_api.json")["api_key"]

### Check classifications

In [18]:
url = "https://api.harvardartmuseums.org/classification"

url_params = {
    "apikey": api_key,
}

resp = requests.get(url, params = url_params)
print(resp.status_code)

200


In [20]:
classifications = []

n = int(resp.json()['info']['pages']) # getting the page number 
        
for i in range(n):
    url_params["page"] = i
    print(f"page {i}")
    
    resp = requests.get(url, params = url_params)

    try: 
        classifications.extend(resp.json()['records']) # add it to the list
    except:
        print(f"Error on page {i+1}") # let me know if there's an error

page 0
page 1
page 2
page 3
page 4
page 5


In [22]:
classifications = pd.DataFrame(classifications)

In [29]:
classifications.sort_values(by = 'objectcount', ascending = False)[0:10]

,objectcount,name,id,lastupdate,classificationid
59,84315,Photographs,17,2020-09-15T04:29:44-0400,17
20,27702,Drawings,21,2020-09-15T04:29:44-0400,21
15,6978,Paintings,26,2020-09-15T04:29:44-0400,26
5,6978,Paintings,26,2020-09-15T04:29:44-0400,26
21,6244,Sculpture,30,2020-09-15T04:29:44-0400,30
35,6190,Vessels,57,2020-09-15T04:29:44-0400,57
49,5898,Seals,189,2020-09-15T04:29:44-0400,189
43,4880,Straus Materials,959,2020-09-15T04:29:44-0400,959
25,4800,Fragments,94,2020-09-15T04:29:44-0400,94
48,4388,Manuscripts,185,2020-09-15T04:29:44-0400,185


## Collecting Full Data
It seems like paintings alone could give us enough data for now. I'll collect all paintings data and download each images.

In [166]:
url = "https://api.harvardartmuseums.org/object"

url_params = {
    "apikey": api_key, 
    "classification": 'Paintings'
}

resp = requests.get(url, params = url_params)

resp.status_code

200

In [ ]:
# getting the number of pages
n = int(resp.json()['info']['pages'])
full_db = []

for i in range(n):
    url_params['page'] = i
    resp = requests.get(url, params = url_params)
    try: 
        full_db.extend(resp.json()['records'])
    except:
        print(f"error on page {i+1}")
              
    pause = np.random.randint(1, 30)
    print(f'{i+1}/{n} complete. pausing for {pause} secs ...')
    time.sleep(pause) # buffer

1/589 complete. pausing for 22 secs ...
2/589 complete. pausing for 25 secs ...
3/589 complete. pausing for 17 secs ...
4/589 complete. pausing for 2 secs ...


In [91]:
full_df = pd.DataFrame(full_db)

In [102]:
image_id_set = full_df[['id', 'primaryimageurl']]

In [161]:
# downloading images
def download_img(df): 
    url = str(df['primaryimageurl'])
    
    if url.startswith('http'):
        try:
            resp = requests.get(df['primaryimageurl']).content
            fpath = f"IMAGES/{df['id']}.jpg"
            with open(fpath, 'wb') as fp:
                fp.write(resp)
        
            img = Image.open(fpath)
            img.thumbnail((500, 500))
            img.save(fpath)
            df['fname'] = fpath
        except:
            df['fname'] = 'error'
    else:
        df['fname'] = None
    
    pause = np.random.randint(1, 10)
    print(f'{df.index()} : pausing for {pause} secs ...')
    time.sleep(pause) # buffer
    
    return df

In [162]:
image_id_set = image_id_set.apply(lambda x: download_img(x), axis = 1)

TypeError: 'Index' object is not callable